# 💾 Module 2: Checkpointers (Persistence)
**Objective:** Solve the "Amnesia" problem. Enable the bot to remember users across different interactions using `MemorySaver`.


In [1]:

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# Import the Checkpointer
from langgraph.checkpoint.memory import MemorySaver

### Step 1: Define the Graph (Smart Version)
We reuse the "SmartState" from Module 1.

In [3]:
# 1. State
class ChatState(TypedDict):
    messages: Annotated[list, add_messages]

# 2. Node
def chatbot_node(state: ChatState):
    # Echo back what the user said to prove we have memory
    last_msg = state["messages"][-1].content
    return {"messages": [AIMessage(content=f"I remember you said: '{last_msg}'")]}

# 3. Builder
builder = StateGraph(ChatState)
builder.add_node("chatbot", chatbot_node)

builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

### Step 2: Add Persistence
We initialize `MemorySaver` and pass it to the `.compile()` method.

In [4]:
# Initialize Memory
memory = MemorySaver()

# Compile with Checkpointer
graph = builder.compile(checkpointer=memory)

print("✅ Graph compiled with MemorySaver")

✅ Graph compiled with MemorySaver


### Step 3: Run with Thread ID
The `thread_id` acts as a unique session key. 

In [8]:
# Config for Session 1
config = {"configurable": {"thread_id": "session_1"}}

print("--- Turn 1 (User Introduces themselves) ---")
input_1 = {"messages": [HumanMessage(content="My name is Sharad.")]}

# Graph saves state to 'session_1'
res1 = graph.invoke(input_1, config=config)


print(res1["messages"][-1].content)



--- Turn 1 (User Introduces themselves) ---
I remember you said: 'My name is Sharad.'


In [10]:
config = {"configurable": {"thread_id": "session_1"}}
print("\n--- Turn 2 (New Execution) ---")
# NOTE: We DO NOT pass the name again. The graph pulls it from memory.
input_2 = {"messages": [HumanMessage(content="What is my name?")]}

res2 = graph.invoke(input_2, config=config)

# Let's inspect the full history to prove it's there
print(f"\nFull History in Memory ({len(res2['messages'])} items):")
for m in res2['messages']:
    print(f" - {m.content}")


--- Turn 2 (New Execution) ---

Full History in Memory (12 items):
 - My name is Sharad.
 - I remember you said: 'My name is Sharad.'
 - What is my name?
 - I remember you said: 'What is my name?'
 - My name is Sharad.
 - I remember you said: 'My name is Sharad.'
 - What is my name?
 - I remember you said: 'What is my name?'
 - My name is Sharad.
 - I remember you said: 'My name is Sharad.'
 - What is my name?
 - I remember you said: 'What is my name?'
